# MarketCap-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an MarketCap-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd 
import requests
import xlsxwriter
import math 

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
5,ABC
6,ABMD
7,ABT
8,ACN
9,ADBE


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'CANSA TL(SA EST MQDENGEKRB)LO/GLA', 'calculationPrice': 'close', 'open': None, 'openTime': None, 'openSource': 'licoffia', 'close': None, 'closeTime': None, 'closeSource': 'faolifci', 'high': None, 'highTime': None, 'highSource': None, 'low': None, 'lowTime': None, 'lowSource': None, 'latestPrice': 131.07, 'latestSource': 'Close', 'latestTime': 'December 16, 2020', 'latestUpdate': 1621458125493, 'latestVolume': None, 'iexRealtimePrice': 131.912, 'iexRealtimeSize': 3, 'iexLastUpdated': 1635791476796, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 128.3, 'previousVolume': 162462097, 'change': -0.07, 'changePercent': -0.00056, 'volume': None, 'iexMarketPercent': 50.43420395798977, 'iexVolume': 1648649, 'avgTotalVolume': 98265221, 'iexBidPrice'

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our MarketCap-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price = data['latestPrice']
market_cap = data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns = ['Ticker', 'Stock Price', 'Market Cap', "Number of Shares to Buy"]
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Cap,Number of Shares to Buy


In [7]:
final_dataframe.append(
    pd.Series(
        [
            symbol,
            price,
            market_cap,
            'N/A'
        ],
        index = my_columns,
    ),
    ignore_index = True
)

,Ticker,Stock Price,Market Cap,Number of Shares to Buy
0,AAPL,131.07,2262913159939,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [8]:
final_dataframe = pd.DataFrame(columns = my_columns)

for ticker in stocks['Ticker'][:5]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{ticker}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()

    price = data['latestPrice']
    market_cap = data['marketCap']
    
    final_dataframe = final_dataframe.append(
    pd.Series(
        [
            ticker,
            price,
            market_cap,
            'N/A'
        ],
        index = my_columns,
    ),
    ignore_index = True
)
final_dataframe

,Ticker,Stock Price,Market Cap,Number of Shares to Buy
0,A,118.38,36858065386,N/A
1,AAL,17.59,10339571314,N/A
2,AAP,162.97,11281583681,N/A
3,AAPL,130.68,2244526030127,N/A
4,ABBV,107.05,185204516046,N/A


In [9]:
final_dataframe

,Ticker,Stock Price,Market Cap,Number of Shares to Buy
0,A,118.38,36858065386,N/A
1,AAL,17.59,10339571314,N/A
2,AAP,162.97,11281583681,N/A
3,AAPL,130.68,2244526030127,N/A
4,ABBV,107.05,185204516046,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [10]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [11]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
market_cap_total = 0
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        price = data[symbol]['quote']['latestPrice']
        market_cap = data[symbol]['quote']['marketCap']
        market_cap_total += market_cap
        final_dataframe = final_dataframe.append(
        pd.Series(
            [
                symbol,
                price,
                market_cap,
                'N/A'
            ],
            index = my_columns,
        ),
        ignore_index = True
        )
        
market_cap_total
    

34775052065099

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [28]:
portfolio_size = input('Enter the value of your portfolio:')
val = float(portfolio_size)

Enter the value of your portfolio:1000000


In [29]:
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    market_cap =  final_dataframe.loc[i, 'Market Cap']
    price =  final_dataframe.loc[i, 'Stock Price']
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(market_cap/market_cap_total*val/price)

final_dataframe = final_dataframe.sort_values(by='Number of Shares to Buy', ascending = False)
final_dataframe

,Ticker,Stock Price,Market Cap,Number of Shares to Buy
3,AAPL,133.95,2181938565685,468.0
198,GE,11.32,98588219489,250.0
56,BAC,29.94,250833739736,240.0
320,MSFT,227.26,1738531611485,219.0
429,T,30.35,217481057503,206.0
365,PFE,38.46,219611101942,164.0
99,CMCSA,52.40,243068770136,133.0
270,KO,55.28,230837819359,120.0
475,VZ,61.06,256303173648,120.0
115,CSCO,45.11,189703563110,120.0


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [14]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: